# Semantic Segmentation Input

## Install Dependencies

In [1]:
!pip install -r requirements.txt

## Setup Imports

In [2]:
import numpy as np
from auxiliary.nifti.io import read_nifti
from rich import print as pprint
from panoptica import (
    ConnectedComponentsInstanceApproximator,
    NaiveThresholdMatching,
    Panoptic_Evaluator,
    SemanticPair,
)

## Load Data

To demonstrate we use a reference and predicition of spine a segmentation without instances.

![semantic_figure](figures/semantic.png)

In [3]:
ref_masks = read_nifti("./spine_seg/semantic/ref.nii.gz")
pred_masks = read_nifti("./spine_seg/semantic/pred.nii.gz")

## Run Evaluation

In [4]:
sample = SemanticPair(pred_masks, ref_masks)

evaluator = Panoptic_Evaluator(
    expected_input=SemanticPair,
    instance_approximator=ConnectedComponentsInstanceApproximator(),
    instance_matcher=NaiveThresholdMatching(),
)

result, debug_data = evaluator.evaluate(sample)

────────────────────────────────────────── Thank you for using panoptica ──────────────────────────────────────────

Please support our development by citing

https://github.com/BrainLesion/panoptica#citation -- Thank you!

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Panoptic: Start Evaluation
-- Got SemanticPair, will approximate instances
-- Got UnmatchedInstancePair, will match instances
-- Got MatchedInstancePair, will evaluate instances
-- evaluate took 6.207059383392334 seconds to execute.


## Inspect Results
The results object allows access to individual metrics and provides helper methods for further processing

In [5]:
# print all results
print(result)

Number of instances in reference (num_ref_instances): 87
Number of instances in prediction (num_pred_instances): 89
True Positives (tp): 73
False Positives (fp): 16
False Negatives (fn): 14
Recognition Quality (rq): 0.8295454545454546
Global Binary Dice (global_bin_dsc): 0.9731641527805414
Global Binary Centerline Dice (global_bin_cldsc): 0.9623567366028574
Segmentation Quality IoU (sq): 0.7940127477906024 +- 0.11547745015679488
Panoptic Quality IoU (pq): 0.6586696657808406
Segmentation Quality Dsc (sq_dsc): 0.8802182546605446 +- 0.07728416427007168
Panoptic Quality Dsc (pq_dsc): 0.7301810521615881
Segmentation Quality Assd (sq_assd): 0.2057371092494465 +- 0.1398348236766068



In [6]:
# get specific metric, e.g. pq
pprint(f"{result.pq=}")

result.pq=0.6586696657808406

In [7]:
# get dict for further processing, e.g. for pandas
pprint("results dict: ", result.to_dict())

results dict: 
{
    'num_ref_instances': 87,
    'num_pred_instances': 89,
    'tp': 73,
    'fp': 16,
    'fn': 14,
    'rq': 0.8295454545454546,
    'global_bin_dsc': 0.9731641527805414,
    'global_bin_cldsc': 0.9623567366028574,
    'sq': 0.7940127477906024,
    'sq_std': 0.11547745015679488,
    'pq': 0.6586696657808406,
    'sq_dsc': 0.8802182546605446,
    'sq_dsc_std': 0.07728416427007168,
    'pq_dsc': 0.7301810521615881,
    'sq_assd': 0.2057371092494465,
    'sq_assd_std': 0.1398348236766068
}